# re_train_09 → weekday_weight 생성 파이프라인 (train만으로)

**목표**
- `re_train_09.csv` (또는 `retrain_09.csv`)만 있을 때, 메뉴×요일별 `weekday_weight`를 **train에서 계산한 실제 값**으로 확정하고,
- 그 값을 **train 파일 맨 끝 컬럼**으로 붙인 `re_train_09_with_weight.csv`를 생성한다.
- 동시에, 동일 수치를 test에 그대로 붙일 수 있도록 매핑 테이블 `weekday_weight_map.csv`도 만든다.

**가중치 정의(편차 기반, 1~2 스케일)**
- 각 메뉴에 대해: `편차 = (요일별 평균) - (해당 메뉴 전체 평균)`
- 메뉴 내부 min-max 정규화: `weekday_weight = 1 + (dev - dev_min) / (dev_max - dev_min)`
- 모든 요일이 동일해서 `dev_max == dev_min`인 경우, 해당 메뉴의 모든 요일 가중치는 `1.0`으로 설정
- (0 값은 절대 나오지 않으며, 후속 단계에서 test에 붙일 때 매칭 실패분의 fallback도 1.0)

---


In [ ]:
import os, glob
import numpy as np
import pandas as pd

## 1) Train 로드 (`re_train_09.csv` 혹은 `retrain_09.csv`)

In [ ]:
# 두 파일명 모두 대응
possible_names = ["re_train_09.csv", "retrain_09.csv"]
train_path = None
for name in possible_names:
    if os.path.exists(name):
        train_path = name
        break

if train_path is None:
    raise FileNotFoundError("re_train_09.csv / retrain_09.csv 중 하나가 현재 작업 디렉토리에 있어야 합니다.")

print("Using train file:", train_path)
df_train = pd.read_csv(train_path)
print("Shape:", df_train.shape)
df_train.head()

## 2) 유틸 함수: 요일 컬럼 생성

In [ ]:
def add_weekday_col(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['영업일자'] = pd.to_datetime(out['영업일자'])
    out['요일'] = out['영업일자'].dt.dayofweek.map({0:'월',1:'화',2:'수',3:'목',4:'금',5:'토',6:'일'})
    return out

## 3) Train에서 (메뉴×요일) → weekday_weight 계산 (값 확정)

In [ ]:
def build_weight_map_from_train(df_train: pd.DataFrame) -> pd.DataFrame:
    """
    입력 df_train에는 '영업일자', '영업장명_메뉴명', '매출수량' 컬럼이 있어야 한다.
    산출물은 (영업장명_메뉴명, 요일, weekday_weight) 매핑 테이블.
    """
    df = add_weekday_col(df_train)

    # 메뉴×요일 평균
    by_wk = (df.groupby(['영업장명_메뉴명','요일'], as_index=False)['매출수량']
               .mean()
               .rename(columns={'매출수량':'wk_mean'}))

    # 메뉴 전체 평균
    by_menu = (df.groupby('영업장명_메뉴명', as_index=False)['매출수량']
                 .mean()
                 .rename(columns={'매출수량':'overall_mean'}))

    merged = by_wk.merge(by_menu, on='영업장명_메뉴명', how='left')
    merged['dev'] = merged['wk_mean'] - merged['overall_mean']  # 방향성 있는 편차

    # 메뉴 내부에서 1~2 스케일
    def scale_1_2(g):
        mn = g['dev'].min()
        mx = g['dev'].max()
        if np.isclose(mx, mn):
            g['weekday_weight'] = 1.0  # 전부 동일하면 1.0
        else:
            g['weekday_weight'] = 1 + (g['dev'] - mn) / (mx - mn)
        return g

    weighted = merged.groupby('영업장명_메뉴명', group_keys=False).apply(scale_1_2)
    weight_map = weighted[['영업장명_메뉴명','요일','weekday_weight']].copy()

    # 안전 검사 (0 금지, NaN 금지)
    if (weight_map['weekday_weight'] <= 0).any():
        raise ValueError("weekday_weight에 0 이하 값이 포함되었습니다. 로직을 점검하세요.")
    if weight_map['weekday_weight'].isna().any():
        raise ValueError("weekday_weight에 NaN이 포함되었습니다.")

    return weight_map

weight_map = build_weight_map_from_train(df_train)
weight_map.head()

## 4) Train에 동일 값 부착 및 저장

In [ ]:
def attach_train_weight(df_any: pd.DataFrame, weight_map: pd.DataFrame) -> pd.DataFrame:
    df = add_weekday_col(df_any)
    out = df.merge(weight_map, on=['영업장명_메뉴명','요일'], how='left')

    # 혹시라도 매칭에서 누락된 경우 1.0으로 대체 (0 금지)
    out['weekday_weight'] = out['weekday_weight'].fillna(1.0)

    # 요일 제거, weight 맨 끝
    out = out.drop(columns=['요일'])
    cols = [c for c in out.columns if c != 'weekday_weight'] + ['weekday_weight']
    return out[cols]

df_train_out = attach_train_weight(df_train, weight_map)

# 저장 (소수 16자리 고정)
train_out_path = "re_train_09_with_weight.csv"
map_out_path = "weekday_weight_map.csv"
df_train_out.to_csv(train_out_path, index=False, float_format='%.16f')
weight_map.to_csv(map_out_path, index=False, float_format='%.16f')

train_out_path, map_out_path, df_train_out.shape, weight_map.shape

## 5) Sanity Check (예시 확인)

In [ ]:
# 예: 임의의 메뉴/요일('일') 가중치 확인
if not df_train.empty:
    sample_menu = df_train['영업장명_메뉴명'].iloc[0]
    sample_day = '일'
    wm_sample = weight_map.query("영업장명_메뉴명 == @sample_menu and 요일 == @sample_day")
    display(wm_sample)
else:
    print("Train is empty.")

## (선택) TEST 파일에 동일 값 부착 예시
테스트 파일이 현재 폴더에 없으면 이 섹션은 아무 작업도 하지 않습니다.

In [ ]:
def attach_weight_to_tests_in_folder(folder_glob: str, weight_map: pd.DataFrame, outdir: str = "test_with_weight"):
    os.makedirs(outdir, exist_ok=True)
    test_files = glob.glob(folder_glob, recursive=True)
    if not test_files:
        print("No TEST files matched:", folder_glob)
        return []

    saved = []
    for path in test_files:
        df_test = pd.read_csv(path)
        df_test_out = attach_train_weight(df_test, weight_map)
        base = os.path.basename(path).replace(".csv", "_with_weight.csv")
        out_path = os.path.join(outdir, base)
        df_test_out.to_csv(out_path, index=False, float_format='%.16f')
        print("Saved:", out_path)
        saved.append(out_path)
    return saved

# 예시: 현재 경로/하위 경로에서 TEST_*_processed.csv 찾기 (없으면 PASS)
_ = attach_weight_to_tests_in_folder("./**/TEST_*_processed.csv", weight_map, outdir="test_with_weight")